In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = []
import csv
with open('arabic_names.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        words.append(row[0].strip())
del words[0]

In [4]:
len(words)

1405

In [5]:
# building the vocabulary and mapping
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}

In [6]:
block_size = 3

X, Y = [], []

for w in words:
    #print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '------>', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


In [13]:
X.shape, X.dtype, Y.shape, Y.dtype


(torch.Size([7646, 3]), torch.int64, torch.Size([7646]), torch.int64)

In [16]:
Y[0]

tensor(8)

In [8]:
g = torch.Generator().manual_seed(123456789)

C = torch.randn((38, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 38), generator=g)
b2 = torch.rand(38, generator=g)
params = [C, W1, b1, W2, b2]

In [9]:
sum(p.nelement() for p in params)

4614

In [45]:
cx = C[X]

cx[1]

tensor([[-0.3775, -0.5169],
        [-0.3775, -0.5169],
        [ 0.4483, -0.5859]], grad_fn=<SelectBackward0>)

In [10]:
for p in params:
    p.requires_grad = True


In [11]:
for _ in range(1000):
    # minibatching
    ix = torch.randint(0, X.shape[0], (14,))
    # Forward pass
    emb = C[X[ix]] # (14, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (14, 100)
    logit = h @ W2 + b2 # (14, 38
    loss = F.cross_entropy(logit, Y[ix])
    # print(loss.item())
    # backward pass
    for p in params:
        p.grad = None
    loss.backward()
    for p in params:
        p.data += -0.1 * p.grad


print(loss.item())


2.954375743865967


In [12]:
emb = C[X] # (14, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (14, 100)
logit = h @ W2 + b2 # (14, 38
loss = F.cross_entropy(logit, Y)
loss.item()

2.9109601974487305

In [122]:
Y

tensor([ 8,  9, 11, 20,  8, 32,  0,  8,  9, 11, 34,  8, 13,  0])